In [47]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision
from torchvision import models
from torchvision import transforms
from PIL import Image

%matplotlib inline

In [48]:
# check if CUDA is available
train_on_gpu = False
# uncomment if you want to us the GPU
# train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


In [88]:
# download & load resnet
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()
model = model.eval()

# move model to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

In [89]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [90]:
# You need to download the following dataset:
# Kaggle Cats and Dogs Dataset
# https://www.microsoft.com/en-us/download/details.aspx?id=54765

# Download the class labels from imagenet
# https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

In [91]:
class CatDogDataset(Dataset):
    def __init__(self, file_list, label, transform):
        self.file_list = file_list
        self.transform = transform
        self.label = label
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(self.file_list[idx]).convert('RGB')
        img = self.transform(img)
        img = img.numpy()
        return img.astype('float32'), self.label

In [92]:
# image transformation
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
transform_normalize = transforms.Normalize(mean.tolist(), std.tolist()) 

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transform_normalize
])

In [93]:
# use only N images
N = 1000
bs = 100

# generate the dataset
cat_dir = 'dogs_cats_dataset/PetImages/Cat/*.jpg'
dog_dir = 'dogs_cats_dataset/PetImages/Dog/*.jpg'
cat_files = glob.glob(cat_dir)
dog_files = glob.glob(dog_dir)

cats = CatDogDataset(cat_files, 0, transform)
dogs = CatDogDataset(dog_files, 1, transform)

catdogs = ConcatDataset([cats, dogs])
dataloader = DataLoader(catdogs, batch_size = bs, shuffle=True)

In [99]:
# generate features
feats = []
targets = []
for idx, (data, target) in enumerate(dataloader):
    print('Batch index: ', idx)

    # move data to gpu
    if train_on_gpu:
        data = data.cuda()
        target = target.cuda()
    
    # generate features
    feats.append( model(data).cpu().detach().numpy() )
    targets.append( target.cpu().detach().numpy() )
    
    if (idx+1) * bs >= N:
        break

Batch index:  0
Batch index:  1
Batch index:  2
Batch index:  3
Batch index:  4
Batch index:  5
Batch index:  6
Batch index:  7
Batch index:  8
Batch index:  9


In [125]:
# save the dataset
np_feats = np.concatenate(feats)
np_targets = np.concatenate(targets)
np_dataset = np.hstack([np_feats, np_targets.reshape(-1,1)])

np.savetxt('dog_cat.csv', np_dataset, delimiter = ',')

In [123]:
np_dataset.shape

(1000, 513)